In [1]:
from keras.applications import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img

from keras.optimizers import Adam, SGD

# Keras imports
from keras.models import  Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Activation, GlobalAveragePooling2D
from keras.callbacks import EarlyStopping
from keras.layers.convolutional import Conv2D, MaxPooling2D

from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator

import glob

from loader_bot import LoaderBot
from splitter import get_skfold_data

import pandas as pd
import numpy as np
import argparse
import cv2

/home/brian/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# https://github.com/DeepLearningSandbox/DeepLearningSandbox/blob/master/transfer_learning/fine-tune.py

IM_WIDTH, IM_HEIGHT = 299, 299 #fixed size for InceptionV3
NB_EPOCHS = 3
BAT_SIZE = 32
FC_SIZE = 1024
NB_IV3_LAYERS_TO_FREEZE = 172

In [7]:
def setup_to_transfer_learn(model, base_model, lr=0.0001):
    """Freeze all layers and compile the model"""
    
    for layer in base_model.layers:
        layer.trainable = False
    
    model.compile(optimizer=SGD(lr=lr, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
    

def add_new_last_layer(base_model, nb_classes):
    """Add last layer to the convnet
    Args:
    base_model: keras model excluding top
    nb_classes: # of classes
    Returns:
    new keras model with last layer
    """
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(FC_SIZE, activation='relu')(x) #new FC layer, random init
    
    predictions = Dense(nb_classes, activation='softmax')(x) #new softmax layer
    
    model = Model(inputs=base_model.input, outputs=predictions)
    
    return model

def setup_to_finetune(model):
    """Freeze the bottom NB_IV3_LAYERS and retrain the remaining top layers.
    note: NB_IV3_LAYERS corresponds to the top 2 inception blocks in the inceptionv3 arch
    Args:
    model: keras model
    """
    for layer in model.layers[:NB_IV3_LAYERS_TO_FREEZE]:
        layer.trainable = False
        
    for layer in model.layers[NB_IV3_LAYERS_TO_FREEZE:]:
        layer.trainable = True
        
    model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
    

def get_nb_files():
    file_paths = glob.glob("../data/stage1_imgs/*.jpg")
    
    return file_paths

def y_labels(file_paths):
    train_img_files = get_nb_files()

    y = np.zeros(len(nb_train_files), dtype=np.int16) # lol int8 overflowed at 128th label resulting in -128

    for idx, file in enumerate(train_img_files):
        # ex file: '../data/stage1_imgs/5766_86.jpg'
        y[idx] = int(file.split("/")[-1].split(".")[0].split("_")[-1])
        
    return y, train_img_files

def give_labels_get_dummies(y):
    '''
    y comes in as a list of ints from 1 to 128
    
    returns one hot matrix of the y values
    '''
    return pd.get_dummies(y.loc[:, "y"])

In [8]:
data_link_dict = get_skfold_data()

# Parameters for Generators
params = {'dim': (299,299),
          'batch_size': 64,
          'n_classes': 128,
          'n_channels': 3,
          'shuffle': False}

# Datasets
X_train_img_paths = data_link_dict["X_train_1"]
y_train = data_link_dict["y_train_1"]

X_test_img_paths = data_link_dict["X_test_1"]
y_test = data_link_dict["y_test_1"]




In [9]:
# Generators
training_generator = LoaderBot(X_train_img_paths, y_train, **params)
validation_generator = LoaderBot(X_test_img_paths, y_test, **params)


In [ ]:
# setup model
base_model = InceptionV3(weights='imagenet', include_top=False) #include_top=False excludes final FC layer
model = add_new_last_layer(base_model, 128)

# transfer learning
setup_to_transfer_learn(model, base_model, lr=0.01)

history_tl = model.fit_generator(generator=training_generator,
                                 validation_data=validation_generator,
                                 epochs=2,
                                 use_multiprocessing=True,
                                 workers=6)

Epoch 1/2
2570/2904 [=========================>....] - ETA: 1:20 - loss: 1.5509 - acc: 0.5890